# Entity linking

You are provided with the data from a knowledge graph and asked to annotate an input document using a general entity linking pipeline approach, consisting of mention detection, candidate selection, and disambiguation steps. 

In [ ]:
import ipytest
import pytest

ipytest.autoconfig()

## 1) Mention detection

You are given an excerpt from a surface form dictionary in the format `SF_DICT[mention][entity] = count`, where `count` refers to the number of times `mention` was linked to `entity` in a given training corpus.
The total count of linked occurrences of a mention is given as the key `_total` (i.e., this is the number of times mention was linked to any entity in the training corpus).
Note that not all linked entities are listed in the dictionary, hence the counts do not necessarily sum up to `_total`.

In [ ]:
SF_DICT = {
    '1992 elections': {
        'wikipedia:Philippine_general_election,_1992': 9,
        'wikipedia:Angolan_presidential_election,_1992': 1,
        '_total': 98
    },
    'angola': {
        'wikipedia:Angola': 4026,
        'wikipedia:Angola_(Portugal)': 6,
        'wikipedia:Angola_national_football_team': 120,
        '_total': 4298
    },
    'democracy': {
        'wikipedia:Democracy': 108,
        'wikipedia:Democracy_(album)': 3,
        '_total': 2162
    },
    'multiparty democracy': {
        'wikipedia:multiparty_democracy': 11,
        '_total': 11
    },
    'one party': {
        'wikipedia:Non-possessors': 1,
        'wikipedia:Single-party_state': 5,
        '_total': 983
    }
}

In [ ]:
TEXT = (
    'Angola changed from a one-party Marxist-Leninist system '
    'ruled by the MPLA to a formal multiparty democracy '
    'following the 1992 elections'
).lower()

We perform mention detection based on the following heuristic:

  - At each term position
    - Start with the longest possible n-gram (n = 8). 
    - If the n-gram is found in the dictionary, the mention and the corresponding entities are kept (and the shorter n-grams are ignored). Otherwise, we try to match the (n-1)-grams. 
    - Repeat until a mention is found or n reaches 0.

In [ ]:
def detect_mentions(text, sf_dict):
    """Performs mention detection in text against a given surface form dictionary.
    
    Args:
        text: Input text.
        sf_dict: Surface form dictionary.
    
    Returns:
        List of matches as `(pos, mention, entity)` tuples ordered by pos, mention, and entity.
        (Term positions are indexed from 0.)
    """
    matches = []
    # TODO: Complete
    return matches

Tests.

In [ ]:
%%run_pytest[clean]

def test_detect_mentions():
    assert detect_mentions(TEXT, SF_DICT) == [
        (0, 'angola', 'wikipedia:Angola'),
        (0, 'angola', 'wikipedia:Angola_(Portugal)'),
        (0, 'angola', 'wikipedia:Angola_national_football_team'),
        (14, 'multiparty democracy', 'wikipedia:multiparty_democracy'),
        (15, 'democracy', 'wikipedia:Democracy'),
        (15, 'democracy', 'wikipedia:Democracy_(album)'),
        (18, '1992 elections', 'wikipedia:Angolan_presidential_election,_1992'),
        (18, '1992 elections', 'wikipedia:Philippine_general_election,_1992')
    ]

## 2) Entity ranking

Entity ranking is based on the commonness score:

$$Commonness(e, m) = p(e|m) = \frac{n(m, e)}{\sum_{e'} n(m, e')}$$

where $n(m, e)$ denotes the number of times entity $e$ is the link target of mention $m$.

In [ ]:
def commonness(entity, mention, sf_dict):
    """Computes the commonness for an entity-mention pair given a surface form dictionary.
    
    Args:
        entity: Entity.
        mention: Mention.
        sf_dict: Surface form dictionary (containing entity-mention count statistics).
        
    Returns:
        Commonness (float).    
    """
    # TODO: Complete
    return None

In [ ]:
def rank_entities(mentions, sf_dict, k=5):
    """Ranks candidate entities for each mention based on commonness and retains 
    the top-k highest-scoring entities for each mention.
    
    Args:
        mentions: Detected mentions (list of `(mention, entity, pos)` tuples).
        sf_dict: Surface form dictionary.
        k: Number of top-scoring entities to keep for each mention.
        
    Returns:
        Candidate entities with scores for each mention. Each mention is 
        represented as a dict `{'mention': xxx, 'pos': yyy, 'entities': zzz`,
        where entities is a list of `(entity, score)` tuples ordered by score desc.
    """
    mentions_entities = []
    # TODO: Complete
    return mentions_entities

Tests.

In [ ]:
%%run_pytest[clean]

@pytest.mark.parametrize("mention,entity,correct_value", [
    ('1992 elections', 'wikipedia:Philippine_general_election,_1992', 9/98),
    ('1992 elections', 'wikipedia:Angolan_presidential_election,_1992', 1/98),
    ('angola', 'wikipedia:Angola', 4026 / 4298),
    ('angola', 'wikipedia:Angola_national_football_team', 120 / 4298),
    ('democracy', 'wikipedia:Democracy', 108/2162),
    ('democracy', 'wikipedia:Democracy_(album)', 3/2162),
    ('multiparty democracy', 'wikipedia:multiparty_democracy', 1)    
])
def test_commonness(entity, mention, correct_value):    
    assert commonness(entity, mention, SF_DICT) == pytest.approx(correct_value, rel=1e-3)
    
def test_rank_entities():
    mentions = detect_mentions(TEXT, SF_DICT)
    ranked_entities = rank_entities(mentions, SF_DICT, k=2)
    assert ranked_entities[0] == {'mention': 'angola',
                                  'pos': 0,
                                  'entities': [
                                      ('wikipedia:Angola', 0.9367147510469986),
                                      ('wikipedia:Angola_national_football_team', 0.02791996277338297)
                                  ]
                                 }
    assert ranked_entities[1] == {'mention': 'multiparty democracy',
                                  'pos': 14,
                                  'entities': [('wikipedia:multiparty_democracy', 1.0)]}

## 3) Disambiguation

Perform disambiguation by simply returning the entity for each mention with the highest score and only if it is above the given threshold.

In case of containment or overlapping mentions, keep only the one with the higher score.

In [ ]:
def disambiguate(ranked_entities, threshold=0.1):
    """Disambiguates entities for each mention by keeping only the highest-scoring one.
    
    Args:
        ranked_entities: List of mentions along with a ranked list of candidate entities.
        threshold: Score threshold
    
    Returns:
        Entity annotations as a list of `(pos, mention, entity)` tuples.
    """
    linked_entities = []
    # TODO: Complete
    return linked_entities

Tests.

In [ ]:
%%run_pytest[clean]

def test_disambiguate():
    mentions = detect_mentions(TEXT, SF_DICT)
    ranked_entities = rank_entities(mentions, SF_DICT)
    linked_entities = disambiguate(ranked_entities, threshold=0.01)
    assert linked_entities == [
        (0, 'angola', 'wikipedia:Angola'),
        (14, 'multiparty democracy', 'wikipedia:multiparty_democracy'),
        (18, '1992 elections', 'wikipedia:Philippine_general_election,_1992')
    ]

## Feedback

Please give (anonymous) feedback on this exercise by filling out [this form](https://forms.gle/2jPayczbFhEcC9K68).